In [2]:
# import Libraries
import pandas as pd
from sqlalchemy import create_engine, text
import datetime

## Import Sales Data

In [3]:
# Create a dataframe 
df_sales = pd.read_csv('sales_data_sample.csv', encoding = 'latin1')
df_sales.head(5)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Data Understanding 

In [4]:
len(df_sales['ORDERDATE'])

2823

In [5]:
# Check missing values 
df_sales.isnull().sum()

ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE1           0
ADDRESSLINE2        2521
CITY                   0
STATE               1486
POSTALCODE            76
COUNTRY                0
TERRITORY           1074
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64

#### Understand the fact granularity

In [6]:
# understand the fact granularity 
df_sales.groupby('ORDERNUMBER').size()

ORDERNUMBER
10100     4
10101     4
10102     2
10103    16
10104    13
         ..
10421     2
10422     2
10423     5
10424     6
10425    13
Length: 307, dtype: int64

In [7]:
# check records where order_number = 10100
df_sales [df_sales['ORDERNUMBER'] == 10100 ]

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
578,10100,30,100.00,3,5151.00,1/6/2003 0:00,Shipped,1,1,2003,...,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Medium
680,10100,50,67.80,2,3390.00,1/6/2003 0:00,Shipped,1,1,2003,...,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Medium
1267,10100,22,86.51,4,1903.22,1/6/2003 0:00,Shipped,1,1,2003,...,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Small
2024,10100,49,34.47,1,1689.03,1/6/2003 0:00,Shipped,1,1,2003,...,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Small


In [8]:
df_sales.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')

#### Extract sales related data from Sales to populate stg_sales 

In [9]:
df_sales = df_sales [['ORDERNUMBER','ORDERLINENUMBER','PRODUCTCODE', 'ORDERDATE',
                 'CUSTOMERNAME','DEALSIZE','QUANTITYORDERED', 'PRICEEACH','SALES',
                     'CONTACTLASTNAME','CONTACTFIRSTNAME', 'COUNTRY']]
df_sales

,ORDERNUMBER,ORDERLINENUMBER,PRODUCTCODE,ORDERDATE,CUSTOMERNAME,DEALSIZE,QUANTITYORDERED,PRICEEACH,SALES,CONTACTLASTNAME,CONTACTFIRSTNAME,COUNTRY
0,10107,2,S10_1678,2/24/2003 0:00,Land of Toys Inc.,Small,30,95.70,2871.00,Yu,Kwai,USA
1,10121,5,S10_1678,5/7/2003 0:00,Reims Collectables,Small,34,81.35,2765.90,Henriot,Paul,France
2,10134,2,S10_1678,7/1/2003 0:00,Lyon Souveniers,Medium,41,94.74,3884.34,Da Cunha,Daniel,France
3,10145,6,S10_1678,8/25/2003 0:00,Toys4GrownUps.com,Medium,45,83.26,3746.70,Young,Julie,USA
4,10159,14,S10_1678,10/10/2003 0:00,Corporate Gift Ideas Co.,Medium,49,100.00,5205.27,Brown,Julie,USA
...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,15,S72_3212,12/2/2004 0:00,Euro Shopping Channel,Small,20,100.00,2244.40,Freyre,Diego,Spain
2819,10373,1,S72_3212,1/31/2005 0:00,"Oulu Toy Supplies, Inc.",Medium,29,100.00,3978.51,Koskitalo,Pirkko,Finland
2820,10386,4,S72_3212,3/1/2005 0:00,Euro Shopping Channel,Medium,43,100.00,5417.57,Freyre,Diego,Spain
2821,10397,1,S72_3212,3/28/2005 0:00,Alpha Cognac,Small,34,62.24,2116.16,Roulet,Annette,France


In [10]:
# format ORDERDATE into datetime format 
df_sales = df_sales.copy()
# convert column to date time 
df_sales['ORDERDATE'] = pd.to_datetime(df_sales['ORDERDATE'])
# drop time
df_sales['ORDERDATE'] = pd.to_datetime(df_sales['ORDERDATE']).dt.date
df_sales['ORDERDATE'].head()

0    2003-02-24
1    2003-05-07
2    2003-07-01
3    2003-08-25
4    2003-10-10
Name: ORDERDATE, dtype: object

In [11]:
# convert orderdate to int
df_sales['ORDERDATE'] = df_sales['ORDERDATE'].apply(lambda X : int(X.strftime('%Y%m%d')))
df_sales['ORDERDATE']

0       20030224
1       20030507
2       20030701
3       20030825
4       20031010
          ...   
2818    20041202
2819    20050131
2820    20050301
2821    20050328
2822    20050506
Name: ORDERDATE, Length: 2823, dtype: int64

#### Create a connection to SQL Srever

In [3]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)
engine

Engine(mssql+pyodbc://@DB_SERVER/DB_NAME?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes)

#### Upload the Data to the  stg_sales MS SQL Table

##### 1) Truncate the staging table

In [13]:
# turncate sales_staging table 
truncate_sql =  '''Truncate Table stg_sales'''

In [14]:
# execute the truncate 
with engine.connect() as con:
    con.execute(text(truncate_sql))
    con.commit()

##### Upload the Data to SQL Tabl

In [15]:
# column mapping 
column_mapping= {'ORDERNUMBER':'order_number',
                  'ORDERLINENUMBER': 'order_line_number',
                  'PRODUCTCODE':'product_code',
                  'ORDERDATE': 'date_id',
                  'CUSTOMERNAME': 'customer_name',
                  'DEALSIZE': 'deal_size',
                  'QUANTITYORDERED': 'quantity',
                  'PRICEEACH': 'unit_price',
                  'SALES': 'sales',
                  'CONTACTLASTNAME': 'contact_lastname',
                 'CONTACTFIRSTNAME': 'contact_firstname',
                 'COUNTRY': 'country'
                 
                 }

In [16]:
df_sales.rename(columns = column_mapping, inplace =True)

In [17]:
df_sales.head()

,order_number,order_line_number,product_code,date_id,customer_name,deal_size,quantity,unit_price,sales,contact_lastname,contact_firstname,country
0,10107,2,S10_1678,20030224,Land of Toys Inc.,Small,30,95.70,2871.00,Yu,Kwai,USA
1,10121,5,S10_1678,20030507,Reims Collectables,Small,34,81.35,2765.90,Henriot,Paul,France
2,10134,2,S10_1678,20030701,Lyon Souveniers,Medium,41,94.74,3884.34,Da Cunha,Daniel,France
3,10145,6,S10_1678,20030825,Toys4GrownUps.com,Medium,45,83.26,3746.70,Young,Julie,USA
4,10159,14,S10_1678,20031010,Corporate Gift Ideas Co.,Medium,49,100.00,5205.27,Brown,Julie,USA


In [18]:
no_records = df_sales.to_sql('stg_sales', con = engine,schema='dbo', if_exists = 'append', index = False)
print(no_records)

39
